In [1]:
%cd /content/drive/MyDrive/Kirill/Projects/Test task_VK_2

/content/drive/MyDrive/Kirill/Projects/Test task_VK_2


In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error as mae

In [3]:
train_df = pd.read_csv('data_preprocessing/final_train.csv')

X = train_df.loc[:, ~train_df.columns.isin(['score'])]
y = train_df['score']

In [4]:
X.head()

,lat,lon,0,7,8,10,11,12,18,21,...,92,93,107,109,159,242,250,346,347,distance
0,0.724309,0.210423,0.142355,0.291666,0.112751,0.064546,0.354221,0.938658,0.326386,0.440000,...,0.343240,0.411185,0.071107,0.165290,0.035966,0.838258,0.418502,0.039461,0.051389,0.080668
1,0.757334,0.292860,0.094093,0.437761,0.106110,0.056762,0.190334,0.916042,0.114529,0.186667,...,0.637724,0.334141,0.020429,0.184066,0.094357,0.778627,0.418502,0.030799,0.055459,0.052733
2,0.114651,0.192895,0.151723,0.388125,0.178486,0.063225,0.311401,0.934817,0.518635,0.342222,...,0.416398,0.337515,0.020875,0.250473,0.021118,0.828130,0.418502,0.027911,0.077430,0.041041
3,0.921666,0.088886,0.423663,0.570066,0.157089,0.063586,0.249114,0.930858,0.323706,0.253333,...,0.517208,0.516301,0.068177,0.173980,0.072054,0.817692,0.418502,0.030799,0.050685,0.071593
4,0.921134,0.088137,0.489140,0.527301,0.118082,0.059441,0.283522,0.929021,0.347827,0.226667,...,0.481356,0.439895,0.013378,0.191465,0.079407,0.805500,0.418502,0.059673,0.059898,0.012488


# Построение модели

In [9]:
import torch
from dataset import *
from utils import *
from models import *
import torch.nn as nn

In [6]:
k_val = 5

device = 'cuda' if torch.cuda.is_available() else 'cpu'
epochs = 200

count_row_one_k = len(X) // k_val
mae_train_all, mae_val_all = [], []
for i in range(k_val):

  x_train_t = np.concatenate([X[:i * count_row_one_k], X[(i + 1) * count_row_one_k:]], axis=0)
  y_train_t = np.concatenate([y[:i * count_row_one_k], y[(i + 1) * count_row_one_k:]], axis=0)

  x_val_t = X[i * count_row_one_k: (i + 1) * count_row_one_k]
  y_val_t = y[i * count_row_one_k: (i + 1) * count_row_one_k]

  train_data_t = MyDataset(x_train_t, y_train_t)
  train_dataloader_t = DataLoader(train_data_t, batch_size=16, shuffle=False)

  test_data_t = MyDataset(x_val_t.to_numpy(), y_val_t.to_numpy())
  test_dataloader_t = DataLoader(test_data_t, batch_size=16, shuffle=False)

  model = MyModel(50, 64).to(device)

  criterion = nn.MSELoss()
  learning_rate = 0.0001

  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
  mae_train, mae_val = train_model(model, criterion, optimizer, epochs, train_dataloader_t, test_dataloader_t)

  mae_train_all.append(mae_train)
  mae_val_all.append(mae_val)

print('\nMean MAE on the train set k-value: '+ str(sum(mae_train_all) / len(mae_train_all)))
print('Mean MAE on the validation k-value: '+ str(sum(mae_val_all) / len(mae_val_all)))

Выходные данные были обрезаны до нескольких последних строк (5000).

Epoch: 167
Loss: tensor(0.0101, grad_fn=<MseLossBackward0>)
Mean MAE on the train set: 0.07310986228046878
Mean MAE on the test set: 0.07352177378458855


Epoch: 168
Loss: tensor(0.0106, grad_fn=<MseLossBackward0>)
Mean MAE on the train set: 0.07315492360822616
Mean MAE on the test set: 0.07404388888524129


Epoch: 169
Loss: tensor(0.0098, grad_fn=<MseLossBackward0>)
Mean MAE on the train set: 0.073003137880756
Mean MAE on the test set: 0.07450096777234322


Epoch: 170
Loss: tensor(0.0113, grad_fn=<MseLossBackward0>)
Mean MAE on the train set: 0.0732685872383656
Mean MAE on the test set: 0.07465462377056098


Epoch: 171
Loss: tensor(0.0077, grad_fn=<MseLossBackward0>)
Mean MAE on the train set: 0.07316054735452898
Mean MAE on the test set: 0.07424192780103439


Epoch: 172
Loss: tensor(0.0093, grad_fn=<MseLossBackward0>)
Mean MAE on the train set: 0.07319815163650821
Mean MAE on the test set: 0.07388537940688622


Epoc

In [7]:
train_data = MyDataset(X.to_numpy(), y.to_numpy())
train_dataloader = DataLoader(train_data, batch_size=16, shuffle=True)

In [8]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

epochs = 200

model = MyModel(50, 64).to(device)

criterion = nn.MSELoss()
learning_rate = 0.0001

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
final_mae_train = train_model(model, criterion, optimizer, epochs, train_dataloader)

torch.save(model, 'model_ep200')
print(final_mae_train)



Epoch: 0
Loss: tensor(0.0173, grad_fn=<MseLossBackward0>)
Mean MAE on the train set: 0.16220515151870066


Epoch: 1
Loss: tensor(0.0112, grad_fn=<MseLossBackward0>)
Mean MAE on the train set: 0.11852053199638975


Epoch: 2
Loss: tensor(0.0277, grad_fn=<MseLossBackward0>)
Mean MAE on the train set: 0.10924118479298804


Epoch: 3
Loss: tensor(0.0096, grad_fn=<MseLossBackward0>)
Mean MAE on the train set: 0.0996794382911272


Epoch: 4
Loss: tensor(0.0073, grad_fn=<MseLossBackward0>)
Mean MAE on the train set: 0.09493213513663395


Epoch: 5
Loss: tensor(0.0242, grad_fn=<MseLossBackward0>)
Mean MAE on the train set: 0.09295259877421695


Epoch: 6
Loss: tensor(0.0518, grad_fn=<MseLossBackward0>)
Mean MAE on the train set: 0.08934934485529988


Epoch: 7
Loss: tensor(0.0052, grad_fn=<MseLossBackward0>)
Mean MAE on the train set: 0.08432406266195787


Epoch: 8
Loss: tensor(0.0129, grad_fn=<MseLossBackward0>)
Mean MAE on the train set: 0.08298603744472864


Epoch: 9
Loss: tensor(0.0026, grad_f